In [1]:
import pandas as pd
import json
from tqdm.auto import tqdm
from sdmetrics.demos import load_demo
import numpy as np
from sdmetrics.reports.single_table import QualityReport


real_data, synthetic_data, metadata = load_demo(modality='single_table')
column_names = [
            'student_id', 'degree_type', 'start_date', 'second_perc', 'work_experience'
]
#column_names = ['start_date', 'work_experience']
metadata['columns'] = {
            key: val for key, val in metadata['columns'].items() if key in column_names
}

real_data = real_data[column_names]
synthetic_data = synthetic_data[column_names]


In [3]:
from sdmetrics.reports.single_table._properties import ColumnShapes, ColumnPairTrends

cpt = ColumnPairTrends()
#real_data['start_date'].iloc[0] = 0
#real_data['employability_perc'].iloc[2] = 'a'
progress_bar = tqdm(total=len(real_data.columns), desc='Column Shapes')
cpt.get_score(real_data, synthetic_data, metadata, progress_bar)

Column Shapes:   0%|          | 0/5 [00:00<?, ?it/s]

0.779

In [4]:
cpt = ColumnPairTrends()

t_r = cpt._get_processed_data(real_data, metadata)
t_r

,student_id,degree_type,start_date,second_perc,work_experience,start_date_discrete,second_perc_discrete
0,17264,Sci&Tech,1595462400000000000,67.00,False,10,6
1,17265,Sci&Tech,1578700800000000000,79.33,True,10,8
2,17266,Comm&Mgmt,1579996800000000000,65.00,False,10,5
3,17267,Sci&Tech,-9223372036854775808,56.00,False,1,4
4,17268,Comm&Mgmt,1593820800000000000,85.80,False,10,10
...,...,...,...,...,...,...,...
210,17474,Comm&Mgmt,1595808000000000000,80.60,False,10,9
211,17475,Sci&Tech,1579737600000000000,58.00,False,10,4
212,17476,Comm&Mgmt,1579910400000000000,67.00,True,10,6
213,17477,Comm&Mgmt,1579392000000000000,74.00,False,10,7


In [5]:
from sdmetrics.reports.utils import discretize_table_data

t_r_1, t_s_1, _ = discretize_table_data(real_data, synthetic_data, metadata)

In [12]:
real_data

,student_id,degree_type,start_date,second_perc,work_experience
0,17264,Sci&Tech,2020-07-23,67.00,False
1,17265,Sci&Tech,2020-01-11,79.33,True
2,17266,Comm&Mgmt,2020-01-26,65.00,False
3,17267,Sci&Tech,NaT,56.00,False
4,17268,Comm&Mgmt,2020-07-04,85.80,False
...,...,...,...,...,...
210,17474,Comm&Mgmt,2020-07-27,80.60,False
211,17475,Sci&Tech,2020-01-23,58.00,False
212,17476,Comm&Mgmt,2020-01-25,67.00,True
213,17477,Comm&Mgmt,2020-01-19,74.00,False


In [9]:
 column_names = [
    'student_id', 'degree_type', 'start_date', 'second_perc', 'work_experience'
]
real_data, synthetic_data, metadata = load_demo(modality='single_table')


metadata['columns'] = {
    key: val for key, val in metadata['columns'].items() if key in column_names
}

real_data['second_perc'].iloc[2] = 'a'

# Run
column_shape_property = ColumnPairTrends()

expected_message_1 = (
    'Error: ValueError Unable to parse string \"a\" at position 2'
)

expected_message_2 = (
    "Error: ValueError could not convert string to float: 'a'"
)

score = column_shape_property.get_score(real_data, synthetic_data, metadata)

# Assert
details = column_shape_property._details

In [12]:
details['Error']

0    Error: ValueError could not convert string to ...
1                                                 None
2                                                 None
3    Error: TypeError '<=' not supported between in...
4    Error: TypeError '<=' not supported between in...
5                                                 None
Name: Error, dtype: object

In [8]:
pd.testing.assert_series_equal(t_r_1['second_perc'], t_r['second_perc_discrete'], check_names=False)

In [7]:
pd.testing.assert_series_equal(t_r_1['start_date'], t_r['start_date_discrete'], check_names=False)

In [3]:
cpt._columns_discretization_failed

{}

In [4]:
real_data

,student_id,degree_type,start_date,second_perc,work_experience
0,17264,Sci&Tech,2020-07-23,67.00,False
1,17265,Sci&Tech,2020-01-11,79.33,True
2,17266,Comm&Mgmt,2020-01-26,65.00,False
3,17267,Sci&Tech,NaT,56.00,False
4,17268,Comm&Mgmt,2020-07-04,85.80,False
...,...,...,...,...,...
210,17474,Comm&Mgmt,2020-07-27,80.60,False
211,17475,Sci&Tech,2020-01-23,58.00,False
212,17476,Comm&Mgmt,2020-01-25,67.00,True
213,17477,Comm&Mgmt,2020-01-19,74.00,False


In [5]:
cpt._details

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation,Error
0,start_date,second_perc,CorrelationSimilarity,0.985451,0.047353,0.076451,None
1,start_date,work_experience,ContingencySimilarity,NaN,NaN,NaN,Error: UFuncTypeError ufunc 'multiply' cannot ...
2,start_date,degree_type,ContingencySimilarity,NaN,NaN,NaN,Error: UFuncTypeError ufunc 'multiply' cannot ...
3,second_perc,work_experience,ContingencySimilarity,0.734884,NaN,NaN,None
4,second_perc,degree_type,ContingencySimilarity,0.697674,NaN,NaN,None
5,work_experience,degree_type,ContingencySimilarity,0.897674,NaN,NaN,None


In [4]:
cpt = ColumnPairTrends()
t = cpt._get_processed_data(real_data, metadata)
t

,student_id,degree_type,start_date,second_perc,work_experience,start_date_discrete,second_perc_discrete
0,17264,Sci&Tech,1.595462e+18,67.00,False,10,6
1,17265,Sci&Tech,1.578701e+18,79.33,True,1,8
2,17266,Comm&Mgmt,1.579997e+18,65.00,False,2,5
3,17267,Sci&Tech,NaN,56.00,False,11,4
4,17268,Comm&Mgmt,1.593821e+18,85.80,False,9,10
...,...,...,...,...,...,...,...
210,17474,Comm&Mgmt,1.595808e+18,80.60,False,10,9
211,17475,Sci&Tech,1.579738e+18,58.00,False,2,4
212,17476,Comm&Mgmt,1.579910e+18,67.00,True,2,6
213,17477,Comm&Mgmt,1.579392e+18,74.00,False,1,7


In [10]:
cpt = ColumnPairTrends()
t3 = cpt._get_processed_data(synthetic_data.copy(), metadata)
cpt._columns_datetime_conversion_failed

{}

In [5]:
report = QualityReport()
report.generate(real_data, synthetic_data, metadata)
details = report.get_details(property_name='Column Pair Trends')
details

Creating report: 100%|██████████| 4/4 [00:00<00:00, 285.04it/s]


Overall Quality Score: 80.13%

Properties:
Column Shapes: 80.67%
Column Pair Trends: 79.6%


,Column 1,Column 2,Metric,Quality Score,Real Correlation,Synthetic Correlation
0,second_perc,start_date,CorrelationSimilarity,0.985451,0.047353,0.076451
1,degree_type,work_experience,ContingencySimilarity,0.897674,NaN,NaN
2,start_date,work_experience,ContingencySimilarity,0.800000,NaN,NaN
3,degree_type,start_date,ContingencySimilarity,0.851163,NaN,NaN
4,second_perc,work_experience,ContingencySimilarity,0.627907,NaN,NaN
5,degree_type,second_perc,ContingencySimilarity,0.613953,NaN,NaN


In [13]:
t_r = cpt._get_processed_data(real_data, metadata)
t_s = cpt._get_processed_data(synthetic_data, metadata)

In [15]:
t_r

,student_id,degree_type,start_date,second_perc,work_experience,start_date_discrete,second_perc_discrete
0,17264,Sci&Tech,1.595462e+18,67.00,False,10,6
1,17265,Sci&Tech,1.578701e+18,79.33,True,1,8
2,17266,Comm&Mgmt,1.579997e+18,65.00,False,2,5
3,17267,Sci&Tech,NaN,56.00,False,11,4
4,17268,Comm&Mgmt,1.593821e+18,85.80,False,9,10
...,...,...,...,...,...,...,...
210,17474,Comm&Mgmt,1.595808e+18,80.60,False,10,9
211,17475,Sci&Tech,1.579738e+18,58.00,False,2,4
212,17476,Comm&Mgmt,1.579910e+18,67.00,True,2,6
213,17477,Comm&Mgmt,1.579392e+18,74.00,False,1,7


In [22]:
from sdmetrics.column_pairs.statistical import ContingencySimilarity, CorrelationSimilarity

metric = ContingencySimilarity
metric.compute_breakdown(t_r[['second_perc_discrete', 'degree_type']], t_s[['second_perc_discrete', 'degree_type']])

{'score': 0.6976744186046512}

In [18]:
cpt._details

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation,Error
0,start_date,second_perc,CorrelationSimilarity,0.985451,0.047353,0.076451,None
1,start_date,work_experience,ContingencySimilarity,0.586047,NaN,NaN,None
2,start_date,degree_type,ContingencySimilarity,NaN,NaN,NaN,"Error: ValueError Unable to parse string ""Comm..."
3,second_perc,work_experience,ContingencySimilarity,0.734884,NaN,NaN,None
4,second_perc,degree_type,ContingencySimilarity,NaN,NaN,NaN,"Error: ValueError Unable to parse string ""Comm..."
5,work_experience,degree_type,ContingencySimilarity,0.897674,NaN,NaN,None


In [4]:
t, _= cpt._get_processed_data(real_data, synthetic_data, metadata)
t['start_date']

0      1595462400000000000
1      1578700800000000000
2      1579996800000000000
3     -9223372036854775808
4      1593820800000000000
              ...         
210    1595808000000000000
211    1579737600000000000
212    1579910400000000000
213    1579392000000000000
214   -9223372036854775808
Name: start_date, Length: 215, dtype: int64

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation,Error
0,start_date,end_date,CorrelationSimilarity,0.898691,0.999999,0.797380,None
1,start_date,salary,CorrelationSimilarity,0.996426,0.052848,0.059996,None
2,start_date,duration,CorrelationSimilarity,0.792278,-0.454971,-0.039527,None
3,start_date,high_perc,CorrelationSimilarity,0.653505,0.491274,-0.201716,None
4,start_date,high_spec,ContingencySimilarity,NaN,NaN,NaN,"Error: ValueError Unable to parse string ""Comm..."
...,...,...,...,...,...,...,...
115,employability_perc,work_experience,ContingencySimilarity,NaN,NaN,NaN,"Error: KeyError ""['work_experience_discrete'] ..."
116,employability_perc,degree_type,ContingencySimilarity,NaN,NaN,NaN,"Error: ValueError Unable to parse string ""Comm..."
117,mba_perc,work_experience,ContingencySimilarity,NaN,NaN,NaN,"Error: KeyError ""['work_experience_discrete'] ..."
118,mba_perc,degree_type,ContingencySimilarity,NaN,NaN,NaN,"Error: ValueError Unable to parse string ""Comm..."


'Error: ValueError Unable to parse string "Commerce" at position 0'

In [10]:
real_data['employability_perc'] = np.nan * np.ones(len(real_data))
report = QualityReport()
report.generate(real_data, synthetic_data, metadata)
details = report.get_details(property_name='Column Shapes')

Creating report:  50%|█████     | 2/4 [00:00<00:00, 52.31it/s]


ValueError: x and y must have length at least 2.

In [2]:
from sdmetrics.reports.single_table._properties import ColumnShapes

In [7]:
from sdmetrics.reports.single_table._properties import ColumnShapes, ColumnPairTrends

column_shapes = ColumnShapes()
real_data['start_date'].iloc[0] = 0
#real_data['employability_perc'].iloc[2] = 'a'
progress_bar = tqdm(total=len(real_data.columns), desc='Column Shapes')
column_shapes.get_score(real_data, synthetic_data, metadata, progress_bar)

Column Shapes:   0%|          | 0/17 [00:00<?, ?it/s]

0.816

In [2]:
metadata

{'primary_key': 'student_id',
 'columns': {'start_date': {'sdtype': 'datetime',
   'datetime_format': '%Y-%m-%d'},
  'end_date': {'sdtype': 'datetime', 'datetime_format': '%Y-%m-%d'},
  'salary': {'sdtype': 'numerical', 'computer_representation': 'Int64'},
  'duration': {'sdtype': 'numerical', 'computer_representation': 'Int64'},
  'student_id': {'sdtype': 'id', 'regex_format': '\\d{30}'},
  'high_perc': {'sdtype': 'numerical', 'computer_representation': 'Float'},
  'high_spec': {'sdtype': 'categorical'},
  'mba_spec': {'sdtype': 'categorical'},
  'second_perc': {'sdtype': 'numerical', 'computer_representation': 'Float'},
  'gender': {'sdtype': 'categorical'},
  'degree_perc': {'sdtype': 'numerical', 'computer_representation': 'Float'},
  'placed': {'sdtype': 'boolean'},
  'experience_years': {'sdtype': 'numerical',
   'computer_representation': 'Float'},
  'employability_perc': {'sdtype': 'numerical',
   'computer_representation': 'Float'},
  'mba_perc': {'sdtype': 'numerical', 'compu

In [4]:
real_data

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,0,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11 00:00:00,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,a,Mkt&Fin,57.80,25000.0,True,2020-01-26 00:00:00,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04 00:00:00,2020-09-27,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,17474,M,80.60,82.00,Commerce,77.60,Comm&Mgmt,False,0,91.0,Mkt&Fin,74.49,40000.0,True,2020-07-27 00:00:00,2020-10-20,3.0
211,17475,M,58.00,60.00,Science,72.00,Sci&Tech,False,0,74.0,Mkt&Fin,53.62,27500.0,True,2020-01-23 00:00:00,2020-08-04,6.0
212,17476,M,67.00,67.00,Commerce,73.00,Comm&Mgmt,True,1,59.0,Mkt&Fin,69.72,29500.0,True,2020-01-25 00:00:00,2020-08-05,6.0
213,17477,F,74.00,66.00,Commerce,58.00,Comm&Mgmt,False,0,70.0,Mkt&HR,60.23,20400.0,True,2020-01-19 00:00:00,2020-04-20,3.0


In [5]:
from sdmetrics.reports.single_table._properties import ColumnShapes, ColumnPairTrends
format_start_data = metadata['columns']['start_date']['datetime_format']
format_end_data = metadata['columns']['end_date']['datetime_format']
cpt = ColumnPairTrends()
#real_data['start_date'] = pd.to_datetime(real_data['start_date'], format=format_start_data)
#real_data['end_date'] = pd.to_datetime(real_data['end_date'], format=format_end_data)
#real_data['start_date'].iloc[0] = 0
#real_data['employability_perc'].iloc[2] = 'a'
size = int(0.5*len(real_data.columns)*(len(real_data.columns)-1))
progress_bar = tqdm(total=size, desc='Column Pair Trends')
t = cpt.get_score(real_data, synthetic_data, metadata, progress_bar)

Column Pair Trends:   0%|          | 0/136 [00:00<?, ?it/s]

In [6]:
cpt.get_visualization()

In [13]:
column_names = [
    'student_id', 'degree_type', 'start_date', 'second_perc', 'work_experience'
]
real_data = pd.read_csv('sdmetrics/demos/single_table/real.csv')[column_names]
synthetic_data = pd.read_csv('sdmetrics/demos/single_table/synthetic.csv')[column_names]
with open('sdmetrics/demos/single_table/metadata.json', 'r') as f:
    metadata = json.load(f)

metadata['columns'] = {
    key: val for key, val in metadata['columns'].items() if key in column_names
}

real_data['second_perc'].iloc[0] = 'a'

# Run
column_shape_property = ColumnPairTrends()


score = column_shape_property.get_score(real_data, synthetic_data, metadata)

/Users/romain_datacebo/Desktop/SDMetrics/sdmetrics/reports/single_table/_properties/column_pair_trends.py:216: UserWarning:

Unable to discretize 'second_perc'. No column pair trends metric will be calculated between this column and boolean/categorical columns. Encountered Error: ValueError Unable to parse string "a" at position 0

/Users/romain_datacebo/Desktop/SDMetrics/sdmetrics/reports/single_table/_properties/column_pair_trends.py:216: UserWarning:

Unable to compute Column Pair Trends for column ('start_date', 'second_perc'). Encountered Error: UFuncTypeError ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U1')) -> None



In [5]:
column_shapes.get_visualization()

In [22]:
column_names = ['student_id', 'degree_type','start_date','second_perc', 'work_experience']
real_data = pd.read_csv('sdmetrics/demos/single_table/real.csv')[column_names]
synthetic_data = pd.read_csv('sdmetrics/demos/single_table/synthetic.csv')[column_names]
with open('sdmetrics/demos/single_table/metadata.json', 'r') as f:
    metadata = json.load(f)

metadata['columns'] = {key: value for key, value in metadata['columns'].items() if key in column_names}


# Run
column_shape_property = ColumnPairTrends()
score = column_shape_property.get_score(real_data, synthetic_data, metadata)
column_shape_property._details['Synthetic Correlation'].to_list()

10it [00:00, 249.46it/s]


[-0.10478798485156637, nan, nan, nan, nan, nan]

In [ ]:
from sdmetrics.reports.single_table._properties.column_pair_trends import ColumnPairTrends
import re
"""Test the ``get_score`` method when the metrics are raising erros for some columns."""
# Setup
column_names = [
    'student_id', 'degree_type', 'start_date', 'second_perc', 'work_experience'
]
real_data = pd.read_csv('sdmetrics/demos/single_table/real.csv')[column_names]
synthetic_data = pd.read_csv('sdmetrics/demos/single_table/synthetic.csv')[column_names]
with open('sdmetrics/demos/single_table/metadata.json', 'r') as f:
    metadata = json.load(f)

metadata['columns'] = {
    key: val for key, val in metadata['columns'].items() if key in column_names
}

real_data['start_date'].iloc[0] = 0
real_data['degree_type'].iloc[2] = 'a'

# Run
column_shape_property = ColumnPairTrends()

expected_message_1 = re.escape(
    "Unable to compute Column Shape for column 'start_date'. "
    "Encountered Error: TypeError '<' not supported between instances of 'str' and 'int'"
)
expected_message_2 = re.escape(
    "Unable to compute Column Shape for column 'degree_type'. "
    "Encountered Error: TypeError '<' not supported between instances of 'str' and 'float'"
)

score = column_shape_property.get_score(real_data, synthetic_data, metadata)

# Assert
assert re.match(expected_message_1, str(recwarn[0].message))
assert re.match(expected_message_2, str(recwarn[1].message))

details = column_shape_property._details
column_names_nan = list(details.loc[pd.isna(details['Score'])]['Column name'])
assert column_names_nan == ['start_date', 'employability_perc']
assert score == 0.826


In [20]:
column_shape_property._details.columns

Index(['Column 1', 'Column 2', 'Metric', 'Score', 'Real Correlation',
       'Synthetic Correlation'],
      dtype='object')

In [11]:
from sdmetrics.reports.single_table import QualityReport

report = QualityReport()
report.generate(real_data, synthetic_data, metadata)
details = report.get_details(property_name='Column Shapes')
#details.rename(columns={'Quality Score': 'Score'}, inplace=True)
#len(details.loc[details['Column 2'] == 'degree_type'])

Creating report: 100%|██████████| 4/4 [00:00<00:00, 17.44it/s]



Overall Quality Score: 82.21%

Properties:
Column Shapes: 81.56%
Column Pair Trends: 82.87%


In [12]:
details

,Column,Metric,Quality Score
0,second_perc,KSComplement,0.627907
1,high_perc,KSComplement,0.553488
2,degree_perc,KSComplement,0.627907
3,experience_years,KSComplement,0.800000
4,employability_perc,KSComplement,0.781395
5,mba_perc,KSComplement,0.841860
6,salary,KSComplement,0.869155
7,start_date,KSComplement,0.701107
8,end_date,KSComplement,0.768919
9,duration,KSComplement,0.826051


In [13]:
report.get_visualization('Column Shapes')

In [24]:
details

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation
0,second_perc,start_date,CorrelationSimilarity,0.985451,0.047353,0.076451
1,degree_type,work_experience,ContingencySimilarity,0.897674,NaN,NaN
2,start_date,work_experience,ContingencySimilarity,0.809302,NaN,NaN
3,degree_type,start_date,ContingencySimilarity,0.893023,NaN,NaN
4,second_perc,work_experience,ContingencySimilarity,0.665116,NaN,NaN
5,degree_type,second_perc,ContingencySimilarity,0.655814,NaN,NaN


In [6]:
details

,Column 1,Column 2,Metric,Score,Real Correlation,Synthetic Correlation
0,end_date,start_date,CorrelationSimilarity,0.906445,0.397321,0.210212
1,salary,start_date,CorrelationSimilarity,0.968412,0.052848,-0.010328
2,duration,start_date,CorrelationSimilarity,0.833848,-0.454971,-0.122667
3,high_perc,start_date,CorrelationSimilarity,0.943772,0.055928,0.168384
4,second_perc,start_date,CorrelationSimilarity,0.985451,0.047353,0.076451
...,...,...,...,...,...,...
115,degree_type,experience_years,ContingencySimilarity,0.800000,NaN,NaN
116,employability_perc,work_experience,ContingencySimilarity,0.693023,NaN,NaN
117,degree_type,employability_perc,ContingencySimilarity,0.716279,NaN,NaN
118,mba_perc,work_experience,ContingencySimilarity,0.767442,NaN,NaN


In [3]:
report.get_visualization(property_name='Column Pair Trends')

In [4]:
from sdmetrics.reports.single_table._properties.column_pair_trends import ColumnPairTrends

proper = ColumnPairTrends()
proper.get_score(real_data, synthetic_data, metadata)
details_2 = proper._details
len(details_2.loc[details_2['Column 2'] == 'degree_type'])

136it [00:00, 260.80it/s]


15

In [5]:
proper.get_visualization()

In [5]:
proper.get_visualization_2()

                    start_date  end_date  salary  duration  high_perc  \
start_date               1.000     1.000   0.053    -0.455      0.491   
end_date                 1.000     1.000   0.075     0.633      0.491   
salary                   0.053     0.075   1.000     0.035      0.077   
duration                -0.455     0.633   0.035     1.000     -0.072   
high_perc                0.491     0.491   0.077    -0.072      1.000   
second_perc              0.608     0.608   0.035    -0.149      0.511   
degree_perc              0.480     0.480  -0.019    -0.012      0.434   
experience_years         0.261     0.261   0.165     0.203      0.086   
employability_perc       0.128     0.128   0.178    -0.042      0.245   
mba_perc                 0.077     0.077   0.175    -0.035      0.355   

                    second_perc  degree_perc  experience_years  \
start_date                0.608        0.480             0.261   
end_date                  0.608        0.480             0.261  

In [12]:
heatmap.dtypes

start_date            object
end_date              object
salary                object
duration              object
high_perc             object
second_perc           object
degree_perc           object
experience_years      object
employability_perc    object
mba_perc              object
dtype: object

In [19]:
import numpy as np
heatmap = proper._get_correlation_matrix('Real Correlation')
np.round(heatmap, 2)

,start_date,end_date,salary,duration,high_perc,second_perc,degree_perc,experience_years,employability_perc,mba_perc
start_date,1,0.999999,0.052848,-0.454971,0.491274,0.607926,0.4798,0.261207,0.127607,0.076881
end_date,0.999999,1,0.075229,0.632543,0.49119,0.607776,0.479774,0.261454,0.127543,0.076822
salary,0.052848,0.075229,1,0.034669,0.076819,0.03533,-0.019272,0.165081,0.178307,0.175013
duration,-0.454971,0.632543,0.034669,1,-0.072228,-0.14864,-0.011698,0.202665,-0.042394,-0.035133
high_perc,0.491274,0.49119,0.076819,-0.072228,1,0.511472,0.434206,0.086045,0.245113,0.354823
second_perc,0.607926,0.607776,0.03533,-0.14864,0.511472,1,0.538404,0.157279,0.261993,0.388478
degree_perc,0.4798,0.479774,-0.019272,-0.011698,0.434206,0.538404,1,0.091014,0.22447,0.402364
experience_years,0.261207,0.261454,0.165081,0.202665,0.086045,0.157279,0.091014,1,0.046072,0.154234
employability_perc,0.127607,0.127543,0.178307,-0.042394,0.245113,0.261993,0.22447,0.046072,1,0.218055
mba_perc,0.076881,0.076822,0.175013,-0.035133,0.354823,0.388478,0.402364,0.154234,0.218055,1


In [11]:
heatmap

,start_date,end_date,salary,duration,high_perc,second_perc,degree_perc,experience_years,employability_perc,mba_perc
start_date,1,0.999999,0.052848,-0.454971,0.491274,0.607926,0.4798,0.261207,0.127607,0.076881
end_date,0.999999,1,0.075229,0.632543,0.49119,0.607776,0.479774,0.261454,0.127543,0.076822
salary,0.052848,0.075229,1,0.034669,0.076819,0.03533,-0.019272,0.165081,0.178307,0.175013
duration,-0.454971,0.632543,0.034669,1,-0.072228,-0.14864,-0.011698,0.202665,-0.042394,-0.035133
high_perc,0.491274,0.49119,0.076819,-0.072228,1,0.511472,0.434206,0.086045,0.245113,0.354823
second_perc,0.607926,0.607776,0.03533,-0.14864,0.511472,1,0.538404,0.157279,0.261993,0.388478
degree_perc,0.4798,0.479774,-0.019272,-0.011698,0.434206,0.538404,1,0.091014,0.22447,0.402364
experience_years,0.261207,0.261454,0.165081,0.202665,0.086045,0.157279,0.091014,1,0.046072,0.154234
employability_perc,0.127607,0.127543,0.178307,-0.042394,0.245113,0.261993,0.22447,0.046072,1,0.218055
mba_perc,0.076881,0.076822,0.175013,-0.035133,0.354823,0.388478,0.402364,0.154234,0.218055,1


In [9]:
proper._get_correlation_matrix('Synthetic Correlation')

,start_date,end_date,salary,duration,high_perc,second_perc,degree_perc,experience_years,employability_perc,mba_perc
start_date,1,0.79738,0.059996,-0.039527,-0.201716,-0.104788,-0.225623,0.306855,-0.176531,0.000171
end_date,0.79738,1,0.044984,-0.002956,-0.159478,-0.092461,-0.182307,0.297424,-0.186206,0.059937
salary,0.059996,0.044984,1,0.088571,0.163097,0.100098,-0.112772,0.155996,0.031811,-0.09919
duration,-0.039527,-0.002956,0.088571,1,-0.036566,0.044933,0.141651,0.127274,0.133925,0.059484
high_perc,-0.201716,-0.159478,0.163097,-0.036566,1,-0.233,0.156474,-0.031583,0.071897,-0.019715
second_perc,-0.104788,-0.092461,0.100098,0.044933,-0.233,1,0.081369,0.028221,0.142578,0.0481
degree_perc,-0.225623,-0.182307,-0.112772,0.141651,0.156474,0.081369,1,-0.059473,0.059914,0.153392
experience_years,0.306855,0.297424,0.155996,0.127274,-0.031583,0.028221,-0.059473,1,0.015717,0.031989
employability_perc,-0.176531,-0.186206,0.031811,0.133925,0.071897,0.142578,0.059914,0.015717,1,-0.016772
mba_perc,0.000171,0.059937,-0.09919,0.059484,-0.019715,0.0481,0.153392,0.031989,-0.016772,1


In [10]:
proper.get_visualization()

ValueError: Invalid column name for _get_correlation_matrix : 'Real correlation'

In [9]:
len(details_2['Score'].dropna())

136

In [6]:
len(details_2.loc[details_2['Metric'] == 'CorrelationSimilarity'])

48

In [4]:
real_data.shape

(215, 17)

In [5]:
from sdmetrics.single_table import (
    ContingencySimilarity, CorrelationSimilarity, KSComplement, TVComplement)

from sdmetrics.column_pairs.statistical.correlation_similarity import CorrelationSimilarity
t = CorrelationSimilarity.compute_breakdown(
    real_data[['start_date', 'end_date']], synthetic_data[['start_date', 'end_date']])


TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [9]:
details = report.get_details(property_name='Column Pair Trends')['Quality Score'].dropna()
len(details)

120

In [18]:
t1 = pd.to_datetime(real_data['start_date'])
t2 = pd.to_datetime(synthetic_data['start_date'])


pearsonr(t1.astype(int) / 10**9, t2.astype(int) / 10**9)

PearsonRResult(statistic=-0.09052251410529105, pvalue=0.18606850772371628)

In [9]:
from scipy.stats import pearsonr, spearmanr

pearsonr(real_data['start_date'], real_data['end_date'])[0]

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [8]:
from sdmetrics.column_pairs.statistical.contingency_similarity import ContingencySimilarity
from sdmetrics.reports.utils import discretize_table_data

discrete_real, discrete_synthetic, _ = discretize_table_data(
            real_data, synthetic_data, metadata
        )
ContingencySimilarity.compute(discrete_real[['salary', 'mba_spec']], discrete_synthetic[['salary', 'mba_spec']])


0.7906976744186047

In [3]:
from sdmetrics.reports.single_table._properties.column_pair_trends import ColumnPairTrends

proper = ColumnPairTrends()
proper.get_score(real_data, synthetic_data, metadata)


136it [00:00, 191.47it/s]


0.804

In [8]:
proper._details

,Column name 1,Column name 2,Metric,Score,Real correlation,Synthetic correlation
0,start_date,end_date,CorrelationSimilarity,0.898691,0.999999,0.797380
1,start_date,salary,CorrelationSimilarity,0.996426,NaN,NaN
2,start_date,duration,CorrelationSimilarity,0.792278,NaN,NaN
3,start_date,student_id,CorrelationSimilarity,0.792278,-0.026862,-0.037637
4,start_date,high_perc,CorrelationSimilarity,0.653505,0.491274,-0.201716
...,...,...,...,...,...,...
131,employability_perc,work_experience,ContingencySimilarity,0.669767,0.056735,-0.201698
132,employability_perc,degree_type,ContingencySimilarity,0.711628,NaN,NaN
133,mba_perc,work_experience,ContingencySimilarity,0.748837,0.168811,0.099095
134,mba_perc,degree_type,ContingencySimilarity,0.679070,NaN,NaN


In [11]:
details

,Column 1,Column 2,Metric,Quality Score,Real Correlation,Synthetic Correlation
0,end_date,start_date,CorrelationSimilarity,0.906445,0.397321,0.210212
1,salary,start_date,CorrelationSimilarity,0.968412,0.052848,-0.010328
2,duration,start_date,CorrelationSimilarity,0.833848,-0.454971,-0.122667
3,high_perc,start_date,CorrelationSimilarity,0.943772,0.055928,0.168384
4,second_perc,start_date,CorrelationSimilarity,0.985451,0.047353,0.076451
...,...,...,...,...,...,...
115,degree_type,experience_years,ContingencySimilarity,0.800000,NaN,NaN
116,employability_perc,work_experience,ContingencySimilarity,0.669767,NaN,NaN
117,degree_type,employability_perc,ContingencySimilarity,0.711628,NaN,NaN
118,mba_perc,work_experience,ContingencySimilarity,0.748837,NaN,NaN


In [14]:
details.loc[details['Metric'] == 'CorrelationSimilarity']

,Column 1,Column 2,Metric,Quality Score,Real Correlation,Synthetic Correlation
0,end_date,start_date,CorrelationSimilarity,0.906445,0.397321,0.210212
1,salary,start_date,CorrelationSimilarity,0.968412,0.052848,-0.010328
2,duration,start_date,CorrelationSimilarity,0.833848,-0.454971,-0.122667
3,high_perc,start_date,CorrelationSimilarity,0.943772,0.055928,0.168384
4,second_perc,start_date,CorrelationSimilarity,0.985451,0.047353,0.076451
5,degree_perc,start_date,CorrelationSimilarity,0.911824,-0.070506,0.105847
6,experience_years,start_date,CorrelationSimilarity,0.948435,-0.025862,-0.128992
7,employability_perc,start_date,CorrelationSimilarity,0.995286,-0.031763,-0.022336
8,mba_perc,start_date,CorrelationSimilarity,0.979492,-0.040913,-0.081928
9,end_date,salary,CorrelationSimilarity,0.951624,0.075229,-0.021523


In [17]:
details_2.loc[details_2['Metric'] == 'CorrelationSimilarity']['Column name 2'].unique()

array(['end_date', 'salary', 'duration', 'student_id', 'high_perc',
       'second_perc', 'degree_perc', 'experience_years',
       'employability_perc', 'mba_perc'], dtype=object)

In [16]:
real_data.dtypes

student_id              int64
gender                 object
second_perc           float64
high_perc             float64
high_spec              object
degree_perc           float64
degree_type            object
work_experience          bool
experience_years        int64
employability_perc    float64
mba_spec               object
mba_perc              float64
salary                float64
placed                   bool
start_date             object
end_date               object
duration              float64
dtype: object

In [5]:
details_2 = proper._details
details_2.loc[details_2['Column name 1'] == 'salary']


,Column name 1,Column name 2,Metric,Score,Real correlation,Synthetic correlation
31,salary,duration,CorrelationSimilarity,0.973049,NaN,NaN
32,salary,student_id,CorrelationSimilarity,0.973049,NaN,NaN
33,salary,high_perc,CorrelationSimilarity,0.956861,NaN,NaN
34,salary,high_spec,ContingencySimilarity,0.758140,NaN,NaN
35,salary,mba_spec,ContingencySimilarity,0.790698,NaN,NaN
36,salary,second_perc,CorrelationSimilarity,0.967616,NaN,NaN
37,salary,gender,ContingencySimilarity,0.786047,NaN,NaN
38,salary,degree_perc,CorrelationSimilarity,0.953250,NaN,NaN
39,salary,placed,ContingencySimilarity,0.781395,NaN,NaN
40,salary,experience_years,CorrelationSimilarity,0.995457,NaN,NaN


In [4]:
metadata

{'primary_key': 'student_id',
 'columns': {'start_date': {'sdtype': 'datetime',
   'datetime_format': '%Y-%m-%d'},
  'end_date': {'sdtype': 'datetime', 'datetime_format': '%Y-%m-%d'},
  'salary': {'sdtype': 'numerical', 'computer_representation': 'Int64'},
  'duration': {'sdtype': 'numerical', 'computer_representation': 'Int64'},
  'student_id': {'sdtype': 'id', 'regex_format': '\\d{30}'},
  'high_perc': {'sdtype': 'numerical', 'computer_representation': 'Float'},
  'high_spec': {'sdtype': 'categorical'},
  'mba_spec': {'sdtype': 'categorical'},
  'second_perc': {'sdtype': 'numerical', 'computer_representation': 'Float'},
  'gender': {'sdtype': 'categorical'},
  'degree_perc': {'sdtype': 'numerical', 'computer_representation': 'Float'},
  'placed': {'sdtype': 'boolean'},
  'experience_years': {'sdtype': 'numerical',
   'computer_representation': 'Float'},
  'employability_perc': {'sdtype': 'numerical',
   'computer_representation': 'Float'},
  'mba_perc': {'sdtype': 'numerical', 'compu

In [8]:
len(proper._details['Score'].dropna())

93

In [48]:
import numpy as np
pivoted = details.pivot(index='Column 1', columns='Column 2', values='Quality Score', )

symmetric = pivoted.fillna(0) + pivoted.fillna(0).T

# Fill diagonal with 1s
np.fill_diagonal(symmetric.values, 1)
symmetric

,degree_perc,degree_type,duration,employability_perc,end_date,experience_years,gender,high_perc,high_spec,mba_perc,mba_spec,placed,salary,second_perc,start_date,work_experience
degree_perc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
degree_type,NaN,0.000000,0.855814,0.711628,0.795349,0.800000,0.888372,0.525581,0.776744,0.679070,0.925581,0.906977,0.776744,0.613953,0.851163,NaN
duration,NaN,0.855814,0.000000,0.911840,0.650699,0.962304,0.832558,0.982169,0.846512,0.952692,0.860465,0.837209,0.973049,0.903213,0.833848,NaN
employability_perc,NaN,0.711628,0.911840,0.000000,0.918056,0.984823,0.679070,0.913392,0.688372,0.882587,0.720930,0.734884,0.926752,0.940293,0.995286,NaN
end_date,NaN,0.795349,0.650699,0.918056,0.000000,0.937302,0.827907,0.916601,0.800000,0.938288,0.827907,0.781395,0.951624,0.918795,0.906445,NaN
experience_years,NaN,0.800000,0.962304,0.984823,0.937302,0.000000,0.800000,0.941186,0.800000,0.938878,0.800000,0.800000,0.995457,0.935471,0.948435,NaN
gender,NaN,0.888372,0.832558,0.679070,0.827907,0.800000,0.000000,0.548837,0.888372,0.730233,0.939535,0.897674,0.786047,0.632558,0.860465,NaN
high_perc,NaN,0.525581,0.982169,0.913392,0.916601,0.941186,0.548837,0.000000,0.506977,0.812731,0.548837,0.465116,0.956861,0.627764,0.943772,NaN
high_spec,NaN,0.776744,0.846512,0.688372,0.800000,0.800000,0.888372,0.506977,0.000000,0.711628,0.883721,0.897674,0.758140,0.609302,0.837209,NaN
mba_perc,NaN,0.679070,0.952692,0.882587,0.938288,0.938878,0.730233,0.812731,0.711628,0.000000,0.697674,0.748837,0.862899,0.829811,0.979492,NaN


Column 2,degree_type,duration,employability_perc,end_date,experience_years,gender,high_perc,high_spec,mba_perc,mba_spec,placed,salary,second_perc,start_date,work_experience
Column 1,,,,,,,,,,,,,,,
degree_perc,0.581395,0.923325,0.917722,0.934286,0.924756,0.618605,0.861134,0.604651,0.875514,0.627907,0.488372,0.953250,0.771483,0.911824,0.595349
degree_type,NaN,0.855814,0.711628,0.795349,0.800000,0.888372,0.525581,0.776744,0.679070,0.925581,0.906977,0.776744,0.613953,0.851163,0.897674
duration,NaN,NaN,0.911840,0.650699,0.962304,0.832558,0.982169,0.846512,0.952692,0.860465,0.837209,0.973049,0.903213,0.833848,0.832558
employability_perc,NaN,NaN,NaN,0.918056,0.984823,0.679070,0.913392,0.688372,0.882587,0.720930,0.734884,0.926752,0.940293,0.995286,0.669767
end_date,NaN,NaN,NaN,NaN,0.937302,0.827907,0.916601,0.800000,0.938288,0.827907,0.781395,0.951624,0.918795,0.906445,0.809302
experience_years,NaN,NaN,NaN,NaN,NaN,0.800000,0.941186,0.800000,0.938878,0.800000,0.800000,0.995457,0.935471,0.948435,0.688372
gender,NaN,NaN,NaN,NaN,NaN,NaN,0.548837,0.888372,0.730233,0.939535,0.897674,0.786047,0.632558,0.860465,0.934884
high_perc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.506977,0.812731,0.548837,0.465116,0.956861,0.627764,0.943772,0.534884
high_spec,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.711628,0.883721,0.897674,0.758140,0.609302,0.837209,0.902326


In [40]:
report.get_visualization(property_name='Column Pair Trends')

In [39]:
details.loc[details['Column 1'] == details['Column 2']]

,Column 1,Column 2,Metric,Quality Score,Real Correlation,Synthetic Correlation


In [14]:
from sdmetrics.reports.single_table._properties.column_shapes import ColumnShapes

column_shapes = ColumnShapes()
real_data['start_date'].iloc[0] = 0
real_data['employability_perc'].iloc[2] = 'a'
column_shapes.get_score(real_data, synthetic_data, metadata, tqdm)

/Users/romain_datacebo/Desktop/SDMetrics/sdmetrics/reports/single_table/_properties/column_shapes.py:57: UserWarning:

Unable to compute Column Shape for column 'start_date'. Encountered Error: TypeError '<' not supported between instances of 'str' and 'int'



TypeError: update() missing 1 required positional argument: 'self'

In [29]:
column_shapes._details

,Column name,Metric,Score
0,start_date,KSComplement,NaN
1,end_date,KSComplement,0.768919
2,salary,KSComplement,0.869155
3,duration,KSComplement,0.826051
4,high_perc,KSComplement,0.553488
5,high_spec,TVComplement,0.902326
6,mba_spec,TVComplement,0.995349
7,second_perc,KSComplement,0.627907
8,gender,TVComplement,0.939535
9,degree_perc,KSComplement,0.627907


In [33]:
details.loc[details['Column name'] == 'start_date']['Score'].isna()

0    True
Name: Score, dtype: bool

In [32]:
details = column_shapes._details
assert details.loc[details['Column name'] == 'start_date']['Score'].isna()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [10]:
real_data

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,0,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaN,NaN,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,17474,M,80.60,82.00,Commerce,77.60,Comm&Mgmt,False,0,91.0,Mkt&Fin,74.49,40000.0,True,2020-07-27,2020-10-20,3.0
211,17475,M,58.00,60.00,Science,72.00,Sci&Tech,False,0,74.0,Mkt&Fin,53.62,27500.0,True,2020-01-23,2020-08-04,6.0
212,17476,M,67.00,67.00,Commerce,73.00,Comm&Mgmt,True,1,59.0,Mkt&Fin,69.72,29500.0,True,2020-01-25,2020-08-05,6.0
213,17477,F,74.00,66.00,Commerce,58.00,Comm&Mgmt,False,0,70.0,Mkt&HR,60.23,20400.0,True,2020-01-19,2020-04-20,3.0


In [7]:
column_shapes._details

,Column name,Metric,Score
0,start_date,KSComplement,0.701107
1,end_date,KSComplement,0.768919
2,salary,KSComplement,0.869155
3,duration,KSComplement,0.826051
4,high_perc,KSComplement,0.553488
5,high_spec,TVComplement,0.902326
6,mba_spec,TVComplement,0.995349
7,second_perc,KSComplement,0.627907
8,gender,TVComplement,0.939535
9,degree_perc,KSComplement,0.627907


In [3]:
column_shapes.get_visualization()

In [3]:
real_data = pd.DataFrame({'col1': [1, '2', 3]})
synthetic_data = pd.DataFrame({'col1': [4, 5, 6]})
metadata = {'columns': {'col1': {'sdtype': 'numerical'}}}

# Run
property = ColumnShapes()
property._generate_details(real_data, synthetic_data, metadata)

  0%|          | 0/1 [00:00<?, ?it/s]

/Users/romain_datacebo/Desktop/SDMetrics/sdmetrics/reports/single_table/_properties/column_shapes.py:35: UserWarning: Unable to compute Column Shape for column 'col1'. Encountered Error: TypeError '<' not supported between instances of 'str' and 'int'
  warnings.warn(


,Column name,Metric,Score
0,col1,KSComplement,NaN


In [4]:
metadata

{'primary_key': 'student_id',
 'columns': {'start_date': {'sdtype': 'datetime',
   'datetime_format': '%Y-%m-%d'},
  'end_date': {'sdtype': 'datetime', 'datetime_format': '%Y-%m-%d'},
  'salary': {'sdtype': 'numerical', 'computer_representation': 'Int64'},
  'duration': {'sdtype': 'numerical', 'computer_representation': 'Int64'},
  'student_id': {'sdtype': 'id', 'regex_format': '\\d{30}'},
  'high_perc': {'sdtype': 'numerical', 'computer_representation': 'Float'},
  'high_spec': {'sdtype': 'categorical'},
  'mba_spec': {'sdtype': 'categorical'},
  'second_perc': {'sdtype': 'numerical', 'computer_representation': 'Float'},
  'gender': {'sdtype': 'categorical'},
  'degree_perc': {'sdtype': 'numerical', 'computer_representation': 'Float'},
  'placed': {'sdtype': 'boolean'},
  'experience_years': {'sdtype': 'numerical',
   'computer_representation': 'Float'},
  'employability_perc': {'sdtype': 'numerical',
   'computer_representation': 'Float'},
  'mba_perc': {'sdtype': 'numerical', 'compu

In [14]:
import json
import re
from unittest.mock import call, patch

import pandas as pd

from sdmetrics.reports.single_table._properties.column_pair_trends import ColumnPairTrends

@patch('sdmetrics.reports.single_table._properties.column_pair_trends.CorrelationSimilarity.compute_breakdown')
@patch('sdmetrics.reports.single_table._properties.column_pair_trends.ContingencySimilarity.compute_breakdown')
def test__generate_details(contingency_compute_mock, correlation_compute_mock):
    """Test the ``_generate_details`` method."""
    # Setup
    real_data = pd.DataFrame({
        'col1': [1, 2, 3],
        'col2': [False, True, True],
        'col3': ['a', 'b', 'c'],
        'col4': pd.to_datetime(['2020-01-01', '2020-01-02', '2020-01-03'])
    })
    synthetic_data = pd.DataFrame({
        'col1': [1, 2, 3],
        'col2': [False, True, True],
        'col3': ['a', 'b', 'c'],
        'col4': pd.to_datetime(['2020-01-01', '2020-01-02', '2020-01-03'])
    })
    metadata = {
        'columns': {
            'col1': {'sdtype': 'numerical'},
            'col2': {'sdtype': 'boolean'},
            'col3': {'sdtype': 'categorical'},
            'col4': {'sdtype': 'datetime'}
        }
    }

    # Run
    column_shape_property = ColumnPairTrends()
    column_shape_property._generate_details(real_data, synthetic_data, metadata)

    # Assert
    expected_calls_contingency = [
        call(real_data[['col1', 'col2']], synthetic_data[['col1', 'col2']]),
        call(real_data[['col1', 'col3']], synthetic_data[['col1', 'col3']]),
        call(real_data[['col2', 'col3']], synthetic_data[['col2', 'col3']]),
        call(real_data[['col2', 'col4']], synthetic_data[['col2', 'col4']]),
        call(real_data[['col3', 'col4']], synthetic_data[['col3', 'col4']]),
    ]

    correlation_compute_mock.assert_called_once_with(real_data[['col1', 'col4']], synthetic_data[['col1', 'col4']])
    contingency_compute_mock.assert_has_calls(expected_calls_contingency)

test__generate_details()

6it [00:00, 2149.82it/s]


AssertionError: Calls not found.
Expected: [call(   col1   col2
0     1  False
1     2   True
2     3   True,    col1   col2
0     1  False
1     2   True
2     3   True),
 call(   col1 col3
0     1    a
1     2    b
2     3    c,    col1 col3
0     1    a
1     2    b
2     3    c),
 call(    col2 col3
0  False    a
1   True    b
2   True    c,     col2 col3
0  False    a
1   True    b
2   True    c),
 call(    col2       col4
0  False 2020-01-01
1   True 2020-01-02
2   True 2020-01-03,     col2       col4
0  False 2020-01-01
1   True 2020-01-02
2   True 2020-01-03),
 call(  col3       col4
0    a 2020-01-01
1    b 2020-01-02
2    c 2020-01-03,   col3       col4
0    a 2020-01-01
1    b 2020-01-02
2    c 2020-01-03)]
Actual: [call(real_data=   col1   col2
0     1  False
1     6   True
2    11   True, synthetic_data=   col1   col2
0     1  False
1     6   True
2    11   True),
 call().__getitem__('score'),
 call(real_data=   col1 col3
0     1    a
1     6    b
2    11    c, synthetic_d

In [22]:
import pandas as pd

df1 = pd.DataFrame({'col1': [1, 2, 3], 'col2': [4, 5, 6]})
df2 = pd.DataFrame({'col1': [4.2, 2.5, 3], 'col2': [7, 8, 9]})

df3 = pd.concat([df1, df2], axis=1)

In [23]:
df3

,col1,col2,col1,col2
0,1,4,4.2,7
1,2,5,2.5,8
2,3,6,3.0,9


In [25]:
import pandas as pd

df = pd.DataFrame({
    'values': [1.5, 2.3, 3.6, 4.2, 5.8, 6.0, 7.1]
})

df['discrete'] = pd.cut(df['values'], bins=3)
df

,values,discrete
0,1.5,"(1.494, 3.367]"
1,2.3,"(1.494, 3.367]"
2,3.6,"(3.367, 5.233]"
3,4.2,"(3.367, 5.233]"
4,5.8,"(5.233, 7.1]"
5,6.0,"(5.233, 7.1]"
6,7.1,"(5.233, 7.1]"


In [24]:
df3 = df3.loc[:,~df3.columns.duplicated()]
df3


,col1,col2
0,1,4
1,2,5
2,3,6


In [21]:
df3['col1'] = [7, 8, 9]
df3

,col1,col2,col1,col2
0,7,4,7,4
1,8,5,8,5
2,9,6,9,6


In [18]:
duplicates = df3.columns[df3.columns.duplicated(keep=False)]

In [20]:
duplicates = df3.columns[df3.columns.duplicated(keep=False)]

for dup in duplicates:
    # split the dataframe by the duplicate column name
    frames = [df3[df3.columns[df3.columns != dup]], df3[dup]]

    # remove exact duplicate columns
    frames[1] = frames[1].loc[:,~frames[1].T.duplicated(keep='first')]

    # rename remaining duplicates in the second dataframe
    frames[1].columns = pd.io.parsers.ParserBase({'names':frames[1].columns})._maybe_dedup_names(frames[1].columns)

    # concat the dataframes back together
    df3 = pd.concat(frames, axis=1)

df3

AttributeError: module 'pandas.io.parsers' has no attribute 'ParserBase'

In [1]:
from sdmetrics.reports.single_table._properties.column_pair_trends import ColumnPairTrends

import pandas as pd
"""Test the ``_get_processed_data`` method."""
# Setup
real_data = pd.DataFrame({
    'col1': [1, 2, 3],
    'col2': [False, True, True],
    'col3': ['a', 'b', 'c'],
    'col4': pd.to_datetime(['2020-01-01', '2020-01-02', '2020-01-03'])
})
synthetic_data = pd.DataFrame({
    'col1': [4, 5, 6],
    'col2': [False, True, True],
    'col3': ['a', 'b', 'c'],
    'col4': pd.to_datetime(['2020-01-04', '2020-01-05', '2020-01-06'])
})
metadata = {
    'columns': {
        'col1': {'sdtype': 'numerical'},
        'col2': {'sdtype': 'boolean'},
        'col3': {'sdtype': 'categorical'},
        'col4': {'sdtype': 'datetime'}
    }
}

# Run
property = ColumnPairTrends()
processed_real, processed_synthetic = property._get_processed_data(real_data, synthetic_data, metadata)

# Assert
expected_processed_real = pd.DataFrame({
    'col1': [1, 2, 3],
    'col2': [False, True, True],
    'col3': ['a', 'b', 'c'],
    'col4': pd.to_numeric(pd.to_datetime(['2020-01-01', '2020-01-02', '2020-01-03'])),
    'col1_discrete': [0, 4, 9],
    'col4_discrete': [0, 4, 9],
})
expected_processed_synthetic = pd.DataFrame({
    'col1': [4, 5, 6],
    'col2': [False, True, True],
    'col3': ['a', 'b', 'c'],
    'col4': pd.to_numeric(pd.to_datetime(['2020-01-04', '2020-01-05', '2020-01-06']).to_numpy()),
    'col1_discrete': [0, 2, 3],
    'col4_discrete': [0, 1, 1],
})

pd.testing.assert_frame_equal(processed_real, expected_processed_real)
pd.testing.assert_frame_equal(processed_synthetic, expected_processed_synthetic)

AssertionError: DataFrame.iloc[:, 4] (column name="col1_discrete") are different

DataFrame.iloc[:, 4] (column name="col1_discrete") values are different (66.66667 %)
[index]: [0, 1, 2]
[left]:  [0, 1, 3]
[right]: [0, 4, 9]
At positional index 1, first diff: 1 != 4

In [5]:
from sdmetrics.reports.utils import discretize_table_data

real_data = pd.DataFrame({
    'col1': [1, 2, 3],
    'col2': [False, True, True],
    'col3': ['a', 'b', 'c'],
    'col4': pd.to_datetime(['2020-01-01', '2020-01-02', '2020-01-03'])
})
synthetic_data = pd.DataFrame({
    'col1': [4, 5, 6],
    'col2': [False, True, True],
    'col3': ['a', 'b', 'c'],
    'col4': pd.to_datetime(['2020-01-04', '2020-01-05', '2020-01-06'])
})
metadata = {
    'columns': {
        'col1': {'sdtype': 'numerical'},
        'col2': {'sdtype': 'boolean'},
        'col3': {'sdtype': 'categorical'},
        'col4': {'sdtype': 'datetime'}
    }
}

# Run
t = discretize_table_data(real_data,synthetic_data, metadata)
t

(   col1   col2 col3  col4
 0     0  False    a     0
 1     4   True    b     4
 2     9   True    c     9,
    col1   col2 col3  col4
 0   NaN  False    a   NaN
 1   NaN   True    b   NaN
 2   NaN   True    c   NaN,
 {'columns': {'col1': {'sdtype': 'categorical'},
   'col2': {'sdtype': 'boolean'},
   'col3': {'sdtype': 'categorical'},
   'col4': {'sdtype': 'categorical'}}})

In [2]:
processed_synthetic

,col1,col2,col3,col4,col1_discrete,col4_discrete
0,4,False,a,1578096000000000000,NaN,NaN
1,5,True,b,1578182400000000000,NaN,NaN
2,6,True,c,1578268800000000000,NaN,NaN


In [3]:
processed_real

,col1,col2,col3,col4,col1_discrete,col4_discrete
0,1,False,a,1577836800000000000,0,0
1,2,True,b,1577923200000000000,4,4
2,3,True,c,1578009600000000000,9,9


In [5]:
from unittest.mock import call, Mock, patch, ANY

import pandas as pd

from sdmetrics.reports.single_table._properties.column_pair_trends import ColumnPairTrends
from plotly.subplots import make_subplots
from plotly import graph_objects as go


@patch('plotly.subplots.make_subplots')
@patch('plotly.graph_objects.Heatmap')
def test_get_visualization_2(mock_heatmap, mock_make_subplots):
    """Test the ``get_visualization`` method."""
    # Setup
    cpt_property = ColumnPairTrends()
    mock_correlation_matrix = Mock()
    mock__get_heatmap = Mock()
    mock_update_layout = Mock()

    cpt_property._get_correlation_matrix = mock_correlation_matrix
    cpt_property._get_heatmap = mock__get_heatmap
    cpt_property._update_layout = mock_update_layout

    mock_make_subplots.add_trace = Mock()

    mock_correlation_matrix.side_effect = [
        'similarity_correlation',
        'real_correlation',
        'synthetic_correlation',
    ]

    mock__get_heatmap.side_effect = [
        'heatmap1',
        'heatmap2',
        'heatmap3',
    ]

    mock_make_subplots.return_value = Mock()

    # Run
    cpt_property.get_visualization()

    # Assert
    mock_correlation_matrix.assert_any_call('Score')
    mock_correlation_matrix.assert_any_call('Real Correlation')
    mock_correlation_matrix.assert_any_call('Synthetic Correlation')

    mock__get_heatmap.assert_any_call('similarity_correlation', 'coloraxis', ANY, ANY)
    mock__get_heatmap.assert_any_call('real_correlation', 'coloraxis2', ANY, 'synthetic_correlation')
    mock__get_heatmap.assert_any_call('synthetic_correlation', 'coloraxis2', ANY, 'real_correlation')

    mock_update_layout.assert_called_once()

    mock_make_subplots.assert_called_once_with(
        rows=2, cols=2, subplot_titles=['Real vs. Synthetic Similarity', 'Numerical Correlation (Real Data)', 'Numerical Correlation (Synthetic Data)'], specs=[[{'colspan': 2, 'l': 0.26, 'r': 0.26}, None], [{}, {}]]
    )

    mock_make_subplots.return_value.add_trace.assert_any_call('heatmap1', 1, 1)
    mock_make_subplots.return_value.add_trace.assert_any_call('heatmap2', 2, 1)
    mock_make_subplots.return_value.add_trace.assert_any_call('heatmap3', 2, 2)

test_get_visualization_2()


ValueError: 
    Invalid element(s) received for the 'data' property of 
        Invalid elements include: ['heatmap1']

    The 'data' property is a tuple of trace instances
    that may be specified as:
      - A list or tuple of trace instances
        (e.g. [Scatter(...), Bar(...)])
      - A single trace instance
        (e.g. Scatter(...), Bar(...), etc.)
      - A list or tuple of dicts of string/value properties where:
        - The 'type' property specifies the trace type
            One of: ['bar', 'barpolar', 'box', 'candlestick',
                     'carpet', 'choropleth', 'choroplethmapbox',
                     'cone', 'contour', 'contourcarpet',
                     'densitymapbox', 'funnel', 'funnelarea',
                     'heatmap', 'heatmapgl', 'histogram',
                     'histogram2d', 'histogram2dcontour', 'icicle',
                     'image', 'indicator', 'isosurface', 'mesh3d',
                     'ohlc', 'parcats', 'parcoords', 'pie',
               

In [14]:
from sdmetrics.reports.utils import discretize_table_data

real_data = pd.DataFrame({
    'col1': [1, 2, 3],
    'col2': [False, True, True],
    'col3': ['a', 'b', 'c'],
    'col4': pd.to_datetime(['2020-01-01', '2020-01-02', '2020-01-03'])
})
synthetic_data = pd.DataFrame({
    'col1': [4, 5, 6],
    'col2': [False, True, True],
    'col3': ['a', 'b', 'c'],
    'col4': pd.to_datetime(['2020-01-04', '2020-01-05', '2020-01-06'])
})
metadata = {
    'columns': {
        'col1': {'sdtype': 'numerical'},
        'col2': {'sdtype': 'boolean'},
        'col3': {'sdtype': 'categorical'},
        'col4': {'sdtype': 'datetime'}
    }
}

result = discretize_table_data(real_data,synthetic_data, metadata)

In [16]:
result[0]

,col1,col2,col3,col4
0,1,False,a,1
1,6,True,b,6
2,11,True,c,11


In [17]:
real_data = pd.DataFrame({
    'col1': [1, 2, 3],
    'col2': [False, True, True],
    'col3': ['a', 'b', 'c'],
    'col4': pd.to_datetime(['2020-01-01', '2020-01-02', '2020-01-03'])
})

In [19]:
from sdmetrics.utils import is_datetime

c = real_data['col4']
is_datetime(c)

True

In [20]:
c

0   2020-01-01
1   2020-01-02
2   2020-01-03
Name: col4, dtype: datetime64[ns]